## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal((784, 512)))
        self.b1 = tf.Variable(tf.zeros((512,)))
        self.W2 = tf.Variable(tf.random.normal((512, 128)))
        self.b2 = tf.Variable(tf.zeros((128,)))
        self.W3 = tf.Variable(tf.random.normal((128, 10)))
        self.b3 = tf.Variable(tf.zeros((10,)))


    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 784))  # 展平输入
        x = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # 第一个全连接层
        x = tf.nn.relu(tf.matmul(x, self.W2) + self.b2)  # 第二个全连接层
        logits = tf.matmul(x, self.W3) + self.b3  # 第三个全连接层
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1618.1831 ; accuracy 0.07721667
epoch 1 : loss 929.5945 ; accuracy 0.17548333
epoch 2 : loss 659.072 ; accuracy 0.26228333
epoch 3 : loss 534.87036 ; accuracy 0.27885
epoch 4 : loss 407.79868 ; accuracy 0.3587
epoch 5 : loss 391.59 ; accuracy 0.37695
epoch 6 : loss 335.45145 ; accuracy 0.43543333
epoch 7 : loss 302.98886 ; accuracy 0.46343333
epoch 8 : loss 255.73412 ; accuracy 0.5086
epoch 9 : loss 240.87904 ; accuracy 0.52643335
epoch 10 : loss 219.58636 ; accuracy 0.55215
epoch 11 : loss 205.79364 ; accuracy 0.56958336
epoch 12 : loss 190.77077 ; accuracy 0.5874
epoch 13 : loss 180.1987 ; accuracy 0.60366666
epoch 14 : loss 168.98573 ; accuracy 0.61621666
epoch 15 : loss 161.4431 ; accuracy 0.6305
epoch 16 : loss 153.66515 ; accuracy 0.64091665
epoch 17 : loss 147.77254 ; accuracy 0.6522167
epoch 18 : loss 142.15196 ; accuracy 0.6599
epoch 19 : loss 137.57808 ; accuracy 0.6684167
epoch 20 : loss 133.4264 ; accuracy 0.67571664
epoch 21 : loss 129.71585 ; accuracy 0.682